# Problem 1(a).  A collectible card class.

You are tasked with organizing information about a Magic: The Gathering card collection.  You are particularly interested in which cards are most rare, and therefore most valuable. Since the game is often played with decks that focus on a particular color or two, you are also interested in the colors of the cards. 

The MTG JSON website (http://mtgjson.com/) has information about different sets of Magic: The Gathering cards in JSON format.  Using the [Example Card](http://mtgjson.com/#exampleCard) information (scroll down if you don't see it!), create a MagicCard class to store relevant information about Magic cards in Python.  

The main features we are interested in are name, colors, and rarity.  Cards may have multiple colors: i.e. a Blue Black card is both blue AND black.  "Colorless" is not a color.

Consider:

* How should someone initialize an instance of your class?
* What functions should your class have?

Your class may have multiple methods for initialization, but at least one of the should accept the MTG JSON represntation of a card as input.

In [1]:
import json

class MagicCard(object):
    
    def __init__(self, json_card, method="json", name="", colors=[], rarity=""):
        if method == "json":
            self.name = json_card["name"]
            if json_card.get("colors",""):
                self.colors = json_card["colors"]
            else:
                self.colors = ["Colorless"]
            self.rarity = json_card["rarity"]
        else:
            self.name = name
            self.colors = colors
            self.rarity = rarity

    def get_name(self):
        """ return link to Flickr page for photo"""
        return self.name
    
    def get_colors(self):
        """ return link to thumbnail version of photo"""
        return self.colors
    
    def get_rarity(self):
        """ return photo title"""
        return self.rarity
    

In [2]:
with open('DTK.json', encoding='utf-8') as data_file:
    dtk_j = json.loads(data_file.read())

# Problem 1(b). Class for a set of Magic cards.

Download the data for the Dragons of Tarkir (abbreviated as DTK) set from the [Individual Sets](http://mtgjson.com/#individualSets) section of the MTG JSON page.  Create a MagicCardSet class to store relevant information about a set of Magic cards, and create an instance of the class which contains all of the information about the Dragons of Tarkir set.  Your class should use your MTG Set class should store the relevant MTG Cards as instances of your MTG Card class.

In [3]:
import pandas as pd
from collections import Counter

class MagicCardSet(object):
    
    def __init__(self, json_set):
        self.name = json_set["name"]
        self.card_list = [MagicCard(card) for card in json_set["cards"]]
  
    def get_set_name(self):
        return self.name
    
    def get_card_list(self):
        cardList = [card.get_name() for card in self.card_list]
        return(cardList)
        
    
    def get_rare(self):
        common = 0
        uncommon = 0
        rare = 0
        mythicrare = 0
        for card in self.card_list:
            if card.get_rarity() == "Common":
                common += 1
            if card.get_rarity() == "Uncommon":
                uncommon += 1
            if card.get_rarity() == "Rare":
                rare += 1
            if card.get_rarity() == "Mythic Rare":
                mythicrare += 1
        rare_rank = [common, uncommon, rare, mythicrare].sort()
        print("Common:", common, "Uncommon:", uncommon, "Rare:", rare, "Mythic Rare:", mythicrare)
        columns = ('name', 'colors', 'rarity')
        rare_list = []
        for card in self.card_list:
            if card.get_rarity() == "Mythic Rare":
                rare_list.append(dict(zip(columns, [card.get_name(), card.get_colors(), card.get_rarity()])))
        rare_df = pd.DataFrame(rare_list)
        return(rare_df)
    
    def common_colors(self):
        color_list = []
        for card in self.card_list:
            if card.get_rarity() == "Uncommon":
                color_list.append(card.get_colors())
        # make the list 'flat' so it'll be easy to count the occurrence of each color
        flat_color_list = [color for sublist in color_list for color in sublist]
        return(Counter(flat_color_list))
            

In [4]:
DTK = MagicCardSet(dtk_j)
DTK.get_rare()

Common: 101 Uncommon: 80 Rare: 53 Mythic Rare: 15


,colors,name,rarity
0,[White],Ojutai Exemplars,Mythic Rare
1,[Blue],Clone Legion,Mythic Rare
2,[Blue],Shorecrasher Elemental,Mythic Rare
3,[Black],Risen Executioner,Mythic Rare
4,[Red],Descent of the Dragons,Mythic Rare
5,[Red],Dragon Whisperer,Mythic Rare
6,[Green],Deathmist Raptor,Mythic Rare
7,[Green],Shaman of Forgotten Ways,Mythic Rare
8,"[Red, Green]",Dragonlord Atarka,Mythic Rare
9,"[White, Green]",Dragonlord Dromoka,Mythic Rare


# Problem 1(c).  Dragons of Tarkir.

The MTG rarity levels are "Common", "Uncommon", "Rare", and "Mythic Rare". What are the rarest cards in the Dragons of Tarkir set (meaning list all card names that are from the least common rarity level)?  What colors are they associated with? You might be able to guess by the rarity level names which rarity level is least common, but we want you to use code to verify which is least common, and then print the card names and colors of those cards.

* For purposes of this question, do not consider "Basic Land" as a rarity level.

Write an MTG Set class method to answer this question, then create a markdown cell which explains your method and conclusions. 

# Answer

First, I set up a counter for each rarity level. I iterated through the set and counted the number of cards at each level. I saw that the rarest is indeed "Mythic Rare." At first, I had it coded to print out each individual name, colors, and rarity for each "Mythic Rare" card, but decided it would be much cleaner to make a dataframe out of it. With some help from the office hours I created a list of each "Mythic Rare" card and it's associated name, colors, and rarity. Finally, I used pandas to create a dataframe.

# Problem 1(d). Common color for Uncommon cards

Write a class method or Python function to determine which color is most represented among Uncommon cards.  Remember, "colorless" is not a color, and a card that is Blue Black counts as a Blue card and a Black card.

When deciding between a class method or a python function, what was your reasoning? ("X was easier for me to code" is not a good reason!)

In [5]:
DTK.common_colors()

Counter({'Black': 15,
         'Blue': 15,
         'Colorless': 10,
         'Green': 15,
         'Red': 15,
         'White': 15})

It seems as though all of the colors are equally represented in the "Uncommon" cards. I chose to code another class method (rather than a python function) because it allowed me to utilize all of the beautiful methods that I previously coded for this assignment!